In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

In [2]:
df = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/properties_2016.csv')

In [3]:
target = pd.read_csv('C:/Users/Stevens/Desktop/BIA 686/zillow/bia_686/data/train_2016_v2.csv')

In [4]:
joined_data = pd.merge(target,df,on="parcelid",how="left")

In [97]:
#--- Creating two additional columns each for the month and day ---
joined_data.transactiondate = pd.to_datetime(joined_data.transactiondate,format='%Y-%m-%d')

joined_data['transaction_year'] = joined_data.transactiondate.dt.weekday.astype(np.int64)
joined_data['transaction_month'] = joined_data.transactiondate.dt.month.astype(np.int64)
joined_data['transaction_dow'] = joined_data.transactiondate.dt.weekday.astype(np.int64)

#--- Dropping the 'transactiondate' column now ---
joined_data = joined_data.drop('transactiondate', 1)

In [5]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [6]:
props, NAlist = reduce_mem_usage(joined_data)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 42.01335906982422  MB
******************************
Column:  parcelid
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  logerror
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  airconditioningtypeid
dtype before:  float64
dtype after:  uint8
******************************
******************************
Column:  architecturalstyletypeid
dtype before:  float64
dtype after:  uint8
******************************
******************************
Column:  basementsqft
dtype before:  float64
dtype after:  uint16
******************************
******************************
Column:  bathroomcnt
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  bedroomcnt
dtype before:  float64
dtype after:  uint8
******************************
******************************
Co

In [124]:
#for c, dtype in zip(joined_data.columns, joined_data.dtypes):
#    if dtype == np.float64:
#        joined_data[c] = joined_data[c].astype(np.float32)